**Antes de aplicar cualquier técnica de imputación, analiza el tipo de variable (numérica, categórica o temporal) y evalúa el impacto que la imputación puede tener sobre los análisis posteriores o los modelos predictivos.**

> **Funciones destacadas utilizadas**

| **Función / Método** | **Descripción** | **Ejemplo de uso** |
|-----------------------|----------------|--------------------|
| `pd.to_datetime()` | Convierte una columna a formato de fecha, permitiendo el manejo de valores no válidos. | `df['A'] = pd.to_datetime(df['A'], errors='coerce')` |
| `.isnull()` / `.sum()` | Identifica y contabiliza valores faltantes. | `df.isnull().sum()` |
| `.apply()` | Permite aplicar funciones personalizadas a columnas. | `df['A'] = df['A'].apply(lambda x: np.nan if x < 0 else x)` |
| `.fillna()` | Imputa valores faltantes con un valor específico o estadístico. | `df['A'] = df['A'].fillna(df['A'].median())` |
| `.mode()` | Obtiene el valor más frecuente de una columna (moda). | `df['A'].fillna(df['A'].mode()[0])` |
| `.interpolate()` | Rellena valores numéricos faltantes mediante interpolación lineal. | `df['A'] = df['A'].interpolate(method='linear')` |
| `.ffill()` / `.bfill()` | Rellena valores faltantes propagando el último valor válido hacia adelante o hacia atrás. | `df['A'] = df['A'].ffill()` |
| `df.duplicated(keep=False)` | Detecta registros duplicados en el DataFrame, mostrando `True` para todas las repeticiones. | `df[df.duplicated(keep=False)]` |
| `get_close_matches()` | Busca coincidencias aproximadas entre cadenas para detectar errores tipográficos o categorías similares. | `from difflib import get_close_matches`<br>`get_close_matches('Medelin', ['Medellín', 'Bogotá', 'Cali'])` |
| `pd.concat()` | Combina series o DataFrames para comparación o unión de resultados. | `pd.concat([faltantes_antes, faltantes_despues], axis=1)` |
| `missingno` | Visualización gráfica de valores faltantes. | `import missingno as msno` |
| `sklearn.impute` | Imputación avanzada (KNN, media, mediana, constante). | `from sklearn.impute import SimpleImputer` |



## <span style="color:#2F749F;"><strong>Ejercicio 1. Limpieza e imputación de un conjunto de datos mixto</strong></span>

#### <span style="color:#2F749F;"><strong>📋 Instrucciones</strong></span>

1. Crea el DataFrame base **df** ejecutando el siguiente código:
    ```python
    df = pd.DataFrame({
        'Nombre': ['Ana', 'Luis', 'Pedro', None, 'Marta', 'Luis', 'Sofía'],
        'Edad': [25, np.nan, 35, 29, -5, 25, None],
        'Ciudad': ['Bogotá', 'Medellín', None, 'Medellín', 'Cali', 'Bogotá', 'Cali'],
        'Ingreso': [3500, 4800, np.nan, 5200, 5100, np.nan, 4700],
        'FechaIngreso': ['2023-01-01', '2023-01-05', None, '2023-01-10', '2023-01-12', None, '2023-01-15']
    })

2. En un nuevo notebook, aplica las etapas del proceso de limpieza e imputación de datos que consideres necesarias sobre el DataFrame df. Ten encuenta incluir:
    - Código correctamente comentado y ejecutado.
    - Explicaciones breves de cada paso.
    - Resultados visuales o estadísticos que evidencien las transformaciones.

In [89]:
import pandas as pd
import numpy as np

In [90]:
df = pd.DataFrame({
        'Nombre': ['Ana', 'Luis', 'Pedro', None, 'Marta', 'Luis', 'Sofía'],
        'Edad': [25, np.nan, 35, 29, -5, 25, None],
        'Ciudad': ['Bogotá', 'Medellín', None, 'Medellín', 'Cali', 'Bogotá', 'Cali'],
        'Ingreso': [3500, 4800, np.nan, 5200, 5100, np.nan, 4700],
        'FechaIngreso': ['2023-01-01', '2023-01-05', None, '2023-01-10', '2023-01-12', None, '2023-01-15']
    })
print(df)

  Nombre  Edad    Ciudad  Ingreso FechaIngreso
0    Ana  25.0    Bogotá   3500.0   2023-01-01
1   Luis   NaN  Medellín   4800.0   2023-01-05
2  Pedro  35.0      None      NaN         None
3   None  29.0  Medellín   5200.0   2023-01-10
4  Marta  -5.0      Cali   5100.0   2023-01-12
5   Luis  25.0    Bogotá      NaN         None
6  Sofía   NaN      Cali   4700.0   2023-01-15


In [91]:
# Remplazar valores nulos en 'Nombre' por 'No identificado'
df['Nombre'] = df['Nombre'].fillna('No identificado')

In [92]:
# Reemplazar valores negativos en 'Edad' por np.nan
# Si x < 0, devuelve np.nan, sino, devuelve x (el valor original).
df['Edad'] = df['Edad'].apply(lambda x: np.nan if x < 0 else x)

# Imputar 'Edad' con Interpolación Lineal
#Las edades están en un rango entre 25 a 35, por lo que pareciera que no varían demasiado
#Podemos estar hablando de ingreso de dinero = población trabajadora joven
df['Edad'] = df['Edad'].interpolate(method='linear')

print(df)

            Nombre  Edad    Ciudad  Ingreso FechaIngreso
0              Ana  25.0    Bogotá   3500.0   2023-01-01
1             Luis  30.0  Medellín   4800.0   2023-01-05
2            Pedro  35.0      None      NaN         None
3  No identificado  29.0  Medellín   5200.0   2023-01-10
4            Marta  27.0      Cali   5100.0   2023-01-12
5             Luis  25.0    Bogotá      NaN         None
6            Sofía  25.0      Cali   4700.0   2023-01-15


In [93]:
# Convertir 'FechaIngreso' a datetime para que None sean 'NaT'
df['FechaIngreso'] = pd.to_datetime(df['FechaIngreso'], errors='coerce')

#Identificar y eliminar filas con ingreso y fecha de ingreso sin valor (doble nulo):
#se asume que no hubo ingreso si ambos se cumplen entonces no hay información reelevante

# Crear una máscara booleana (True donde se cumple la condición)
# La condición es: (Ingreso es nulo) Y (FechaIngreso es nulo)
condicion_doble_nulo = (df['Ingreso'].isna()) & (df['FechaIngreso'].isna())

# Usar el operador ~ (negación) para seleccionar las filas que no cumplen
# la condición de doble nulo y borra las filas que sí la cumplen.
df_limpieza1 = df[~condicion_doble_nulo].copy()

print(df_limpieza1)

            Nombre  Edad    Ciudad  Ingreso FechaIngreso
0              Ana  25.0    Bogotá   3500.0   2023-01-01
1             Luis  30.0  Medellín   4800.0   2023-01-05
3  No identificado  29.0  Medellín   5200.0   2023-01-10
4            Marta  27.0      Cali   5100.0   2023-01-12
6            Sofía  25.0      Cali   4700.0   2023-01-15


Al eliminar las filas donde no hay valores de ingreos y fecha de ingreso, una de las filas donde no había información de ciudad también fue eliminada. Por otro lado, al imputar los datos de edad antes de eliminar las dos columnas referentes a ingreso, seguíamos teniendo información de otras edades en la población de muestra. Si estamos haciendo una caracterización de ingreso según edad y ciudad, el nombre es una manera de identificar usuarios pero no se vuelve un feature determinante, por lo que dejarlo como 'No identificado' no afectaría el resto de valores.

## <span style="color:#2F749F;"><strong>Ejercicio 2. Limpieza de duplicados</strong></span>

#### <span style="color:#2F749F;"><strong>📋 Instrucciones</strong></span>

1. Crea el DataFrame base **df** ejecutando el siguiente código:
    ```python
    df = pd.DataFrame({
        'ID': [101, 102, 102, 103, 104, 104, 104],
        'Nombre': ['Ana', 'Luis', 'Luis', 'Marta', 'Carlos', 'Carlos', 'Carlos'],
        'Edad': [25, 30, 30, 29, 40, 40, 41],
        'Ciudad': ['Bogotá', 'Cali', 'Cali', 'Medellín', 'Cali', 'Cali', 'Cali'],
        'FechaRegistro': ['2023-01-01', '2023-01-05', '2023-01-05', '2023-01-10',
                        '2023-01-15', '2023-01-15', '2023-01-16']
    })

2.  En el mismo notebook anterior, para el nuevo dataframe df, responde a las siguientes preguntas (utilizando python):
    - ¿Cuál es el total de registros originales?
    - ¿Cuáles y cuántos son los duplicados exactos?
    - ¿Cuáles y cuántos son los duplicados por varias columnas?
    - ¿Cuántos registros debes eliminar?
    - ¿Cuántos registros quedan después de la limpieza?

In [94]:
df = pd.DataFrame({
        'ID': [101, 102, 102, 103, 104, 104, 104],
        'Nombre': ['Ana', 'Luis', 'Luis', 'Marta', 'Carlos', 'Carlos', 'Carlos'],
        'Edad': [25, 30, 30, 29, 40, 40, 41],
        'Ciudad': ['Bogotá', 'Cali', 'Cali', 'Medellín', 'Cali', 'Cali', 'Cali'],
        'FechaRegistro': ['2023-01-01', '2023-01-05', '2023-01-05', '2023-01-10',
                        '2023-01-15', '2023-01-15', '2023-01-16']
    })
print(df)

    ID  Nombre  Edad    Ciudad FechaRegistro
0  101     Ana    25    Bogotá    2023-01-01
1  102    Luis    30      Cali    2023-01-05
2  102    Luis    30      Cali    2023-01-05
3  103   Marta    29  Medellín    2023-01-10
4  104  Carlos    40      Cali    2023-01-15
5  104  Carlos    40      Cali    2023-01-15
6  104  Carlos    41      Cali    2023-01-16


In [95]:
# Total de registros originales
total_registros_originales = len(df)
print(f" Total de registros originales: {total_registros_originales}")

# Duplicados exactos (toda la fila es idéntica)
duplicados_exactos= df.duplicated(keep=False)
duplicados_exactos = df[duplicados_exactos]
total_duplicados = len(duplicados_exactos)
eliminar = df.duplicated(keep='first').sum() #keep='first' para contar solo las repeticiones (el número a eliminar)

print("\nDuplicados exactos:")
print(f"Cantidad de duplicados exactos: {total_duplicados} (Filas 102 y 104)")
print(f"Registros a eliminar (repeticiones): {eliminar}")
print("\nRegistros duplicados exactos:")
print(duplicados_exactos)


# Duplicados por varias columnas ('ID', 'Nombre', 'Ciudad' ya que estas son más distintivas de 1 sola persona)
# Las variables de 'Edad' y 'Fecharegistro' podrían ser repetidas sin necesidad de que sea la misma persona
columnas_clave = ['ID', 'Nombre', 'Ciudad']
duplicados_clave = df.duplicated(subset=columnas_clave, keep=False)
duplicados_clave = df[duplicados_clave]
total_duplicados_clave = len(duplicados_clave)

# Contar cuántas repeticiones se van a eliminar hay basadas en las columnas clave
eliminar_clave = df.duplicated(subset=columnas_clave, keep='first').sum()

print(f"\nDuplicados por columnas clave ({', '.join(columnas_clave)}):")
print(f"Total de ocurrencias de duplicados por clave: {total_duplicados_clave}")
print(f"Registros a eliminar (repeticiones): {eliminar_clave}")
print("\nRegistros duplicados por clave:")
print(duplicados_clave)


# Registros restantes después de la limpieza
# Eliminamos duplicados basados en las variables clave
df_limpio = df.drop_duplicates(subset=columnas_clave, keep='last')
registros_restantes = len(df_limpio)
print(f"\nRegistros que quedan después de la limpieza: {registros_restantes}")

print(df_limpio)

 Total de registros originales: 7

Duplicados exactos:
Cantidad de duplicados exactos: 4 (Filas 102 y 104)
Registros a eliminar (repeticiones): 2

Registros duplicados exactos:
    ID  Nombre  Edad Ciudad FechaRegistro
1  102    Luis    30   Cali    2023-01-05
2  102    Luis    30   Cali    2023-01-05
4  104  Carlos    40   Cali    2023-01-15
5  104  Carlos    40   Cali    2023-01-15

Duplicados por columnas clave (ID, Nombre, Ciudad):
Total de ocurrencias de duplicados por clave: 5
Registros a eliminar (repeticiones): 3

Registros duplicados por clave:
    ID  Nombre  Edad Ciudad FechaRegistro
1  102    Luis    30   Cali    2023-01-05
2  102    Luis    30   Cali    2023-01-05
4  104  Carlos    40   Cali    2023-01-15
5  104  Carlos    40   Cali    2023-01-15
6  104  Carlos    41   Cali    2023-01-16

Registros que quedan después de la limpieza: 4
    ID  Nombre  Edad    Ciudad FechaRegistro
0  101     Ana    25    Bogotá    2023-01-01
2  102    Luis    30      Cali    2023-01-05
3  10

Uno de los usuarios con el mismo ID y nombre que estaba duplicado en todas las catacteristicas, volvía a identificarse cuando hacemos los duplicados por clave, solo cambia la edad de 40 a 41 por lo que puede asumirse como un mismo usuario que generó un error de escritura en la edad.

## <span style="color:#2F749F;"><strong>Ejercicio 3. Corrección de errores tipográficos o de codificación</strong></span>

#### <span style="color:#2F749F;"><strong>📋 Instrucciones</strong></span>

1. Crea el DataFrame base **df** ejecutando el siguiente código:
    ```python
    df = pd.DataFrame({
        'Ciudad': ['bogota', 'Bogotá', 'BOGOTA', 'bogotá', 'bogata', 'Bógota', 'BogoTa', 'Cali', 'calí', 'medellín', 'medellin']
    })

2.  En el mismo notebook anterior, para el nuevo dataframe df, estandariza la variable ciudad utilizando python.

In [96]:
df = pd.DataFrame({
        'Ciudad': ['bogota', 'Bogotá', 'BOGOTA', 'bogotá', 'bogata', 'Bógota', 'BogoTa', 'Cali', 'calí', 'medellín', 'medellin']
    })
print(df)

      Ciudad
0     bogota
1     Bogotá
2     BOGOTA
3     bogotá
4     bogata
5     Bógota
6     BogoTa
7       Cali
8       calí
9   medellín
10  medellin


In [ ]:
%pip install unidecode
import unidecode
from difflib import get_close_matches

In [ ]:
# Definir valores maestros
valores_maestros = ['bogota', 'cali', 'medellin']
umbral = 0.8 # Umbral de similitud para considerar una corrección

def corregir_typo_ciudad(texto_sucio, maestros, umbral): #Función para corregir errores tipográficos en una columna de texto.
    if pd.isnull(texto_sucio):
        return texto_sucio

    # Minusculas y acentos
    texto_limpio = unidecode.unidecode(str(texto_sucio)).lower()

    # Eerrores tipográficos con get_close_matches
    # Busca el valor maestro más cercano
    coincidencia = get_close_matches(texto_limpio, maestros, n=1, cutoff=umbral)

    # Si encuentra una coincidencia por encima del umbral, la devuelve
    if coincidencia:
        return coincidencia[0]
    else:
        # Si no hay coincidencia, devuelve el valor tal cual (o np.nan, si lo prefieres)
        return texto_limpio

# Aplicar funcion a 'Ciudad'
df['Ciudad_Limpia'] = df['Ciudad'].apply(lambda x: corregir_typo_ciudad(x, valores_maestros, umbral))

print(df)